In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Customer_Churn").getOrCreate()

df = spark.read.csv("Data\customer_churn.csv", inferSchema = True, header = True)

# observe structure and some data
df.printSchema()
print("rows:",df.count(), "  cols:", len(df.columns))
df.show(5)
df.describe().show()

"""
    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company 
"""

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)

rows: 900   cols: 10
+----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|           Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|
+----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|
|   Kevin Mu

'\n    Name : Name of the latest contact at Company\n    Age: Customer Age\n    Total_Purchase: Total Ads Purchased\n    Account_Manager: Binary 0=No manager, 1= Account manager assigned\n    Years: Totaly Years as a customer\n    Num_sites: Number of websites that use the service.\n    Onboard_date: Date that the name of the latest contact was onboarded\n    Location: Client HQ Address\n    Company: Name of Client Company \n'

In [3]:
# pick attributes and assemble imput vector
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'],outputCol='features')

data = df
output = assembler.transform(data)
final_data = output.select('features','churn')

 # split
train_churn,test_churn = final_data.randomSplit([0.7,0.3])


In [5]:
# Model building
from pyspark.ml.classification import LogisticRegression
lr_churn = LogisticRegression(labelCol='churn')
fitted_churn_model = lr_churn.fit(train_churn)
training_sum = fitted_churn_model.summary
training_sum.predictions.describe().show()
training_sum.predictions.show(5)

+-------+-------------------+-------------------+
|summary|              churn|         prediction|
+-------+-------------------+-------------------+
|  count|                618|                618|
|   mean|0.15857605177993528|0.11165048543689321|
| stddev| 0.3655761796852578|0.31519106526894414|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[22.0,11254.38,1....|  0.0|[4.31532001000903...|[0.98681392321355...|       0.0|
|[26.0,8787.39,1.0...|  1.0|[0.63921845055844...|[0.65457676892570...|       0.0|
|[28.0,9090.43,1.0...|  0.0|[1.45849787232266...|[0.81130282075497...|       0.0|
|[28.0,11128.95,1....|  0.0|[3.92231583904556...|[0.98058

In [7]:
# Evaluating results
from pyspark.ml.evaluation import BinaryClassificationEvaluator
pred_and_labels = fitted_churn_model.evaluate(test_churn)
pred_and_labels.predictions.show(5)



+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[25.0,9672.03,0.0...|    0|[4.36616585056213...|[0.98745942290303...|       0.0|
|[26.0,8939.61,0.0...|    0|[6.04945816268048...|[0.99764641251192...|       0.0|
|[27.0,8628.8,1.0,...|    0|[5.19113657142871...|[0.99446512788405...|       0.0|
|[28.0,8670.98,0.0...|    0|[7.41471791337430...|[0.99939804029753...|       0.0|
|[29.0,5900.78,1.0...|    0|[4.01492642478546...|[0.98227554298878...|       0.0|
+--------------------+-----+--------------------+--------------------+----------+
only showing top 5 rows



In [8]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                           labelCol='churn')
auc = churn_eval.evaluate(pred_and_labels.predictions)
auc

0.8073578595317725

In [18]:
# using model for prediciton on new data

new_customers = spark.read.csv("Data\\new_customers.csv", inferSchema = True, header = True)
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [20]:
# Assemble new cust data to correct format

test_new_customers = assembler.transform(new_customers)
test_new_customers.printSchema()

# fit model to all the data (not just train)
final_lr_model = lr_churn.fit(final_data)

final_results = final_lr_model.transform(test_new_customers)

final_results.select('Company','prediction').show()
# there are 4 customers that should be account manager assigned to
#among the 6 new customers

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+

